# AISC2009 - Natural Language Processing 01
## In-Class Application Exercise 4
## Student Name & ID
### Nikhil Nikhare - 500197137
### Deepansh Suhane - 500201397
### Muhammad Ahmer - 500196858
### Jaswinder Singh - 500195345
### Syed Jafri - 500196461

# **Text Summarization**

**Step 1** 

Firstly, we retrieve an article from Wikipedia (here we have used Loyalist College Article) in order to summarise them. Then, we have utilised few libraries in order to achieve our goal. The first package that needs to be downloaded is called beautiful soup which is an important tool for web scraping.

In [10]:
 pip install beautifulsoup4


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**Step 2**

Now we have installed lxml library which is used to scrape data from the web and it also helped us to handle XML and HTML files with an ease.

In [11]:
pip install lxml


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**Step 3**

Here we have used bs4 which stands beautiful soup 4 which is the latest version of the library. We have also used urllib.request library as it is an extensible library when opening an URL and re library for importing regular expression.

We will now import an article from Wikipedia as given below.

The idea is to cut down on the amount of time you spend reading (regardless of where the text originates from). 
Hence, we have created a tool that is flexible enough to be used to numerous sources.

In [22]:
import bs4 as bs
import urllib.request
import re

scraped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Loyalist_College')
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

**Step 4**

We will now remove Square Brackets and Extra Spaces.

In [23]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

**Step 5**

Removing Special Characters and Digits.

In [24]:
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

**Step 6**

Now we will import nltk ibrary as it offers us a variety of text processing libraries and helped us in Tokenizing and many other activities.

We have also installed "popular" subset of NLTK data set which helped us to implement our code in a streamlined way. So, if you're unsure which datasets or models you'll require, you must use this subset of NLTK.

In [16]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/deepanshsuhane/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/deepanshsuhane/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/deepanshsuhane/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/deepanshsuhane/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/deepanshsuhane/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/deepanshsuhane/nltk_data...
[nl

True

**Step 7**

We've already preprocessed the data at this point. We will now execute tokenizer package as it helped us to divide a string into sub-strings. These tokens or sub-strings are excellent for identifying patterns and can be considered as the first step in stemming and lemmatization.

In [25]:
sentence_list = nltk.sent_tokenize(article_text)

**Step 8**

Moving forward, we will now remove stop words (Frequently used words like "the," "a," "an," and "in" are examples of stop words).

We also need to find the frequency of occurence of each words. Hence, we have used the formatted_article_text variable so as to determine how frequently each word appears. Since it lacks punctuation, numerals, or any other special characters, we used this variable to determine the frequency of occurrence.

*Understanding our if else loop:-*

To determine if there are any stop words, we have created a loop which runs through each phrase and then the related terms. 
If not, we then determine whether the terms are included in the word frequency dictionary or not. The dictionary adds the word as a key and sets its value to 1 if it is encountered for the first time. 
Alternatively, if the word was already in the dictionary, its value is simply increased by 1.

In [26]:
stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}
for word in nltk.word_tokenize(formatted_article_text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

**Step 9**

*Calculating the weighted frequency:* Here, we have divided the total number of occurrences by the frequency of the term that appears the most frequently to obtain the weighted frequency.

In [27]:
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

**Step 10**

*Calculating Sentence Score:* We have combined the weighted frequencies of the words that appear in each sentence to calculate a sentense score.

*Understanding our second if else loop:-* 

First, a blank sentence_scores dictionary is created. The sentences themselves will serve as the dictionary's keys, and the sentences associated scores will serve as the dictionary's values. The sentences in the sentence_list are then tokenized into words using a loop through each sentence.

The next step is to see if the word is listed in the dictionary word frequencies. This check is made because the word frequencies were calculated using the formatted_article_text_object, which does not contain any stop words, numbers, etc., whereas the sentence list list was constructed from the article_text object.

In [28]:
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

**Step 11**

**Final Step to get the Summary**

Till here we have the sentence_scores dictionary which constitute corresponding score. Now to summarize our article, we took top N scores based on their highest scores.

We will now recall the nlargest function of the heapq library to extract the top seven phrases with the highest scores.

In [29]:
import heapq
summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)
print(summary)

Loyalist College houses some of its students in the Loyalist College Residence located on the college campus. This building contains laundry facilities, computer access, security office, food station and entertainment area. Originally operated out of a local high school, Loyalist College moved to its present 200-acre (81 ha) campus on Wallbridge-Loyalist Road in 1968. Loyalist College (formally Loyalist College of Applied Arts and Technology) is an English-language college in Belleville, Ontario, Canada. The name of the college reflects the area's original settlement by United Empire Loyalists. The entertainment area includes pool tables, ping-pong and television for student use. The college operates a satellite campus in Bancroft, Ontario and is associated with First Nations Technical Institute in the Tyendinaga Mohawk Territory reserve.
